<a href="https://colab.research.google.com/github/Ladvien/lego_id_training_data/blob/master/lego_classifier_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Ladvien/lego_id_training_data.git
!mkdir ./data
!mkdir ./data/output
!ls

Cloning into 'lego_id_training_data'...
remote: Enumerating objects: 5766, done.
remote: Counting objects: 100% (5766/5766), done.
remote: Compressing objects: 100% (5763/5763), done.
remote: Total 5766 (delta 1), reused 5766 (delta 1), pack-reused 0
Receiving objects: 100% (5766/5766), 82.15 MiB | 11.01 MiB/s, done.
Resolving deltas: 100% (1/1), done.
data  lego_id_training_data  sample_data


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 16 06:13:38 2019

@author: ladvien and ezracc
"""
# Import Tensorflow
import tensorflow as tf


# Import needed tools.
import os
import matplotlib.pyplot as plt
import json
import numpy as np
from scipy import stats

# Import Keras
import tensorflow.keras
from tensorflow.keras.layers import Dense,Flatten, Dropout, Lambda
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, MaxPooling2D, Conv2D, Activation
from tensorflow.compat.v1.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image

# Tensorboard
from tensorboard import program
import webbrowser
import time

In [0]:
# https://medium.com/tensorflow/tf-keras-on-tpus-on-colab-674367932aa0
# %load_ext watermark
# %watermark -p tensorflow,numpy -m

In [0]:
#################################
# TODO: Make experiment folder
#################################
# 1. Generate a unique id
# 2. Save weighs to folder
# 3. Save tensorboard logs and open tensorboard to this folder

#################################
# Training Parameters
#################################

continue_training       = False

input_shape             = (300, 300, 3) # This is the shape of the image width, length, colors
image_size              = (input_shape[0], input_shape[1]) # DOH! image_size is (height, width)
train_test_ratio        = 0.2
zoom_range              = 0.2
shear_range             = 0.2

# Hyperparameters
batch_size              = 16
epochs                  = 180
steps_per_epoch         = 100
validation_steps        = 100 
optimizer               = 'adadelta' 
learning_rate           = 1.0
val_save_step_num       = 1

path_to_graphs          = './data/output/logs/'
model_save_dir          = './data/output/'
train_dir               = './lego_id_training_data/train/'
val_dir                 = './lego_id_training_data/test/'

In [0]:
#################################
# Helper functions
#################################

def make_dir(dir_path):
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    

def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history['loss'], label='Train loss')
    ax[0].plot(history.epoch, history.history['val_loss'], label='Validation loss')
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history['acc'], label='Train acc')
    ax[1].plot(history.epoch, history.history['val_acc'], label='Validation acc')
    ax[0].legend()
    ax[1].legend()

#################################
# Create needed dirs
#################################
make_dir(model_save_dir)

In [7]:
#################################
# Data generators
#################################

# These Keras generators will pull files from disk
# and prepare them for training and validation.
augs_gen = ImageDataGenerator (
    shear_range = shear_range,  
    zoom_range = shear_range,        
    horizontal_flip = True,
    validation_split = train_test_ratio
)  

train_gen = augs_gen.flow_from_directory (
    train_dir,
    target_size = image_size, # THIS IS HEIGHT, WIDTH
    batch_size = batch_size,
    class_mode = 'sparse',
    shuffle = True
)

test_gen = augs_gen.flow_from_directory (
    val_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'sparse',
    shuffle = False
)

Found 4453 images belonging to 10 classes.
Found 2715 images belonging to 10 classes.


In [0]:
#################################
# Save Class IDs
#################################
classes_json = train_gen.class_indices
num_classes = len(train_gen.class_indices)

In [0]:
#################################
# Model Building
#################################

with open(model_save_dir + 'classes.json', 'w') as fp:
    json.dump(classes_json, fp, indent = 4)

def ConvBlock(model, num_layers, filters):
    for i in range(num_layers):
        model.add(tensorflow.keras.layers.Conv2D(filters,(3,3), activation = 'selu'))
        model.add(tensorflow.keras.layers.SeparableConv2D(filters, (3, 3), activation = 'selu'))
        model.add(tensorflow.keras.layers.BatchNormalization())
        model.add(tensorflow.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)))
    
def FCN():
    model = tensorflow.keras.models.Sequential()
    model.add(tensorflow.keras.layers.Lambda(lambda x: x, input_shape = input_shape))
    ConvBlock(model, 1, 32)
    ConvBlock(model, 1, 64)
    ConvBlock(model, 1, 128)
    ConvBlock(model, 1, 256)
    model.add(tensorflow.keras.layers.Flatten())
    model.add(tensorflow.keras.layers.Dense(1024, activation = 'selu'))
    model.add(tensorflow.keras.layers.Dropout(0.2))
    model.add(tensorflow.keras.layers.Dense(num_classes, activation = 'softmax'))
    return model

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
# model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# model.add(tf.keras.layers.Dropout(0.25))

# model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
# model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.Dropout(0.25))

# model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
# model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
# model.add(tf.keras.layers.Dropout(0.25))

# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(256))
# model.add(tf.keras.layers.Activation('elu'))
# model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(10))
# model.add(tf.keras.layers.Activation('softmax'))
# model.summary()

In [13]:
#################################
# Create model
#################################

def get_optimizer(optimizer, learning_rate = 0.001):
    if optimizer == 'adam':
        return tensorflow.keras.optimizers.Adam(lr = learning_rate, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0., amsgrad = False)
    elif optimizer == 'sgd':
        return tensorflow.keras.optimizers.SGD(lr = learning_rate, momentum = 0.99) 
    elif optimizer == 'adadelta':
        return tensorflow.keras.optimizers.Adadelta(lr=learning_rate, rho=0.95, epsilon=None, decay=0.0)

selected_optimizer = get_optimizer(optimizer, learning_rate)

model = FCN()
model.summary()

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = selected_optimizer,
    metrics = ['accuracy']
)

W0828 14:26:00.165407 140010922551168 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:4075: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
separable_conv2d (SeparableC (None, 296, 296, 32)      1344      
_________________________________________________________________
batch_normalization_3 (Batch (None, 296, 296, 32)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 144, 144, 64)     

In [14]:
#################################
# Keras Callbacks
#################################
best_model_weights = model_save_dir + 'base.model'

checkpoint = ModelCheckpoint(
    best_model_weights,
    monitor = 'val_loss',
    verbose = 1,
    save_best_only = True,
    mode = 'min',
    save_weights_only = False,
    period = val_save_step_num
)

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto'
)

tensorboard = TensorBoard(
    log_dir = model_save_dir + '/logs',
    histogram_freq=0,
    batch_size=16,
    write_graph=True,
    write_grads=True,
    write_images=False,
)

csvlogger = CSVLogger(
    filename = model_save_dir + 'training.csv',
    separator = ',',
    append = False
)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=40,
    verbose=1, 
    mode='auto',
    cooldown=1 
)

callbacks = [checkpoint, csvlogger, tensorboard]

W0828 14:26:03.091193 140010922551168 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [0]:
# tb = program.TensorBoard()
# tb.configure(argv=[None, '--logdir', '/home/ladvien/lego_sorter/data/output/logs/'])
# url = tb.launch()
# # Wait for load
# time.sleep(1)
# webbrowser.open('http://localhost:6006')

In [0]:
#################################
# Execute Training
#################################

if continue_training:
    model.load_weights(best_model_weights)
    model_score = model.evaluate_generator(test_gen, steps = validation_steps)

    print('Model Test Loss:', model_score[0])
    print('Model Test Accuracy:', model_score[1])


history = model.fit_generator(
    train_gen, 
    steps_per_epoch  = steps_per_epoch, 
    validation_data  = test_gen,
    validation_steps = validation_steps,
    epochs = epochs, 
    verbose = 1,
    callbacks = callbacks
)

# tpu_model = tf.contrib.tpu.keras_to_tpu_model(
#     model,
#     strategy=tf.contrib.tpu.TPUDistributionStrategy(
#         tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
#     )
# )
# tpu_model.compile(
#     optimizer=tf.train.AdamOptimizer(learning_rate=1e-3, ),
#     loss=tf.keras.losses.sparse_categorical_crossentropy,
#     metrics=['sparse_categorical_accuracy']
# )

# show_final_history(history)

Epoch 1/180
 99/100 [============================>.] - ETA: 1s - loss: 15.5442 - acc: 0.3427
Epoch 00001: val_loss improved from inf to 7.03804, saving model to ./data/output/base.model
100/100 [==============================] - 144s 1s/step - loss: 15.4074 - acc: 0.3449 - val_loss: 7.0380 - val_acc: 0.3338
Epoch 2/180
 26/100 [======>.......................] - ETA: 1:10 - loss: 2.4384 - acc: 0.5168

In [46]:
# tpu_model.fit_generator(
#     generator = train_gen
# )

AttributeError: ignored

In [0]:

#################################
# Test Image
#################################

# 1. Get each class and label.
# 2. Generate nubmber of n predictions for each class.
# 3. Take the mode of the predictions of each class.
# 4. Compare the prediction mode against actual class.

number_of_tests = 100
number_correct = 0

for key, item in classes_json.items():    
    class_name = key
    predictions = []
    for i in range(0, number_of_tests):
        img = image.load_img(f'./generated_images/test/{class_name}/{class_name}_{str(i)}.png', target_size = input_shape)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        classes = model.predict_classes(x, batch_size=1)
        predictions.append(classes[0])
    
    
    if int(stats.mode(np.asarray(predictions))[0]) == item:
        print(f'{key} is correct')
        number_correct += 1

print(f'Correct classes: {number_correct}')
#################################
# Evaluate Training
#################################
model.load_weights(best_model_weights)
model_score = model.evaluate_generator(test_gen, steps = validation_steps)

print('Model Test Loss:', model_score[0])
print('Model Test Accuracy:', model_score[1])
    
#################################
# Save Model
#################################
model_json = model.to_json()
with open(model_save_dir + 'model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save(model_save_dir + 'model.h5')
print('Weights Saved')



NameError: ignored

In [0]:
# Use a tf optimizer rather than a Keras one for now
opt = tf.train.AdamOptimizer(learning_rate)
model.compile(
      optimizer=opt,
      loss='categorical_crossentropy',
      metrics=['acc'])

In [0]:
# TPU training.

tpu_model = tensorflow.tpu (
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

NameError: ignored